In [1]:
# !pip install import-ipynb
# import import_ipynb

%tensorflow_version 1.x

!pip install numpy==1.16.1

TensorFlow 1.x selected.
     |████████████████████████████████| 17.3MB 228kB/s 
ERROR: tensorflow 1.15.2 has requirement gast==0.2.2, but you'll have gast 0.3.3 which is incompatible.
ERROR: umap-learn 0.4.6 has requirement numpy>=1.17, but you'll have numpy 1.16.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5


In [1]:

import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt 
from keras.models import Model, Sequential
from keras.layers import Conv1D,Flatten,Conv2D, Dropout, MaxPooling2D, Input,Conv3D, MaxPooling3D, Dense, BatchNormalization
from keras import Input,layers
from keras.layers.merge import concatenate 
from google.colab import drive
from keras.optimizers import Adam,RMSprop
from keras.wrappers.scikit_learn import KerasClassifier,KerasRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
drive.mount('/content/drive')
# %cp -av fromfolder tofolder
# %cd "/content/drive/My Drive/Newdata/"
# from model_functions import *

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# 将441 变成 2*2 (但是2^2其中也是 10^10,10^11,11^10,11^11四象限)

In [2]:

Data1 = np.load('/content/drive/My Drive/Newdata/matrix_output_con.npy') 
Data2 = np.load('/content/drive/My Drive/Newdata/matrix_output_pre.npy') 
Data3 = np.load('/content/drive/My Drive/Newdata/matrix_output_post.npy') 
Data4 = np.load('/content/drive/My Drive/Newdata/matrix_output_fu.npy') 

# print(Data2.shape)  # 将21*21 的矩阵分成 4个部分
def from_441_to_4(Data):

  Y =[]
  for i in range(24):
    X =[]
    X.append(Data[i,:10,:10])
    X.append(Data[i,:10,10:])
    X.append(Data[i,10:,:10])
    X.append(Data[i,10:,10:])


    Y.append(X)
  Tmp = np.array(Y)
  print(Tmp.shape)
  return Tmp

To_4_con =  from_441_to_4(Data1)
To_4_pre =  from_441_to_4(Data2)
To_4_post =  from_441_to_4(Data3)
To_4_fu =  from_441_to_4(Data4) 

print(To_4_fu[0,0])

(24, 4)
(24, 4)
(24, 4)
(24, 4)
[[2. 2. 2. 2. 2. 2. 2. 2. 1. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 1. 2. 2. 2. 1.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]]


# 将10^10,10^11,11^10,11^11四象限 变成四个值(0黑 1灰 2白三种class)

In [ ]:
# 将分成4份的象限融合，每个象限只用一个数值代表。 0：盲，1：部分盲，2：正常 每个象限用最多值代替（例：假如1最多那就是1）
def change_to_4_dots(Data): # 分成的四个矩阵 每个矩阵用其中数值最多的数来表示（0,1,2）
  Tmp = Data[:]   
  for i in range(24):
    for j in range(4):

      a = np.count_nonzero(Data[i,j] == 0) 
      b = np.count_nonzero(Data[i,j] == 1)
      c = np.count_nonzero(Data[i,j] == 2)
      if a == max(a,b,c):
        Tmp[i,j] = 0 
      elif b == max(a,b,c):
        Tmp[i,j] = 1
      elif c == max(a,b,c):
        Tmp[i,j] = 2 
  return Tmp
  
aaa = change_to_4_dots(To_4_con)
bbb = change_to_4_dots(To_4_pre)
ccc = change_to_4_dots(To_4_post)
ddd = change_to_4_dots(To_4_fu)
print(aaa.shape)   #(24, 4)

np.save('/content/drive/My Drive/Newdata/_441to4/To_4_con',aaa)
np.save('/content/drive/My Drive/Newdata/_441to4/To_4_pre',bbb)
np.save('/content/drive/My Drive/Newdata/_441to4/To_4_post',ccc)
np.save('/content/drive/My Drive/Newdata/_441to4/To_4_fu',ddd)



(24, 4)


In [ ]:
# 此cell将 4个label 变成  [4,2]形式   例子 [0 0 1 2] 变成 [1 0 0],[1 0 0], [0 1 0], [0 0 1]

def from_4_to_3_label(data):
  TMP = []
  for i in range(24):
    Tmp =[]
    for j in range(4):
      tmp = []
      if data[i,j] == 0:
        tmp.append(1) #生成 1,0,0存进list
        tmp.append(0)
        tmp.append(0)
        
      elif data[i,j] == 1:
        tmp.append(0)
        tmp.append(1)
        tmp.append(0)
      
      elif data[i,j] == 2:
        tmp.append(0)
        tmp.append(0)
        tmp.append(1)
      # print(tmp)
      Tmp.append(tmp) 
      # print(len(Tmp))      
    TMP.append(Tmp)
  result = np.array(TMP)
  return result

A = from_4_to_3_label(aaa)
B = from_4_to_3_label(bbb)
C = from_4_to_3_label(ccc)
D = from_4_to_3_label(ddd)
print(A.shape)  # (24, 4, 3)

# np.save('/content/drive/My Drive/Newdata/_441to4/To_16_con',A)
# np.save('/content/drive/My Drive/Newdata/_441to4/To_16_pre',B)
# np.save('/content/drive/My Drive/Newdata/_441to4/To_16_post',C)
# np.save('/content/drive/My Drive/Newdata/_441to4/To_16_fu',D)

(24, 4, 3)


In [ ]:
def make_50_times(data):

  tmp = []
  for i in range(24):
    for j in range(50):
      tmp.append(data[i])
  result = np.array(tmp)
  return result

AA = make_50_times(A)
BB = make_50_times(B)
CC = make_50_times(C)
DD = make_50_times(D)
print(AA.shape)  # (1200, 4, 3)

(1200, 4, 3)


In [ ]:
#该每50个就集成一次 按照病人分类， 24个病人 每个病人150个数据（# pre post fu 每50个 ）
for i in range (24):  
  tmp = np.concatenate((BB[i*50:50*(i+1)],CC[i*50:50*(i+1)],DD[i*50:50*(i+1)]),axis=0)
  np.save('/content/drive/My Drive/Newdata/ordered_by_patient/4/output_150_4_3_'+str(i+1),tmp)
  print(i+1)
  print(tmp.shape)

In [ ]:
#该每50个就集成一次 按照病人分类， 24个病人 每个病人200个数据（# con pre post fu 每50个 ）
for i in range (24):  
  tmp = np.concatenate((AA[i*50:50*(i+1)],BB[i*50:50*(i+1)],CC[i*50:50*(i+1)],DD[i*50:50*(i+1)]),axis=0)
  np.save('/content/drive/My Drive/Newdata/ordered_by_patient/4/output_200_4_3_'+str(i+1),tmp)
  print(i+1)
  print(tmp.shape)

# 10^10,10^11,11^10,11^11四象限 变成四个值(0 1 两种class)

In [ ]:
# 将分成4份的象限融合，每个象限只用一个数值代表。 0盲 + 1部分盲 = 盲(用0表示)，2 = 正常(用1表示) 每个象限用最多值代替（例：假如0最多那就是0）
def change_to_4_dots(Data): # 分成的四个矩阵 每个矩阵用其中数值最多的数来表示（0,1）
  Tmp = Data[:]   
  for i in range(24):
    for j in range(4):

      a = np.count_nonzero(Data[i,j] == 0) + np.count_nonzero(Data[i,j] == 1)

      c = np.count_nonzero(Data[i,j] == 2)
      if a == max(a,c):
        Tmp[i,j] = 0 
      elif c == max(a,c):
        Tmp[i,j] = 1
  return Tmp
  
aaa = change_to_4_dots(To_4_con)
bbb = change_to_4_dots(To_4_pre)
ccc = change_to_4_dots(To_4_post)
ddd = change_to_4_dots(To_4_fu)
print(aaa.shape)   #(24, 4)
print(ddd)

# np.save('/content/drive/My Drive/Newdata/_441to4/To_4_con',aaa)
# np.save('/content/drive/My Drive/Newdata/_441to4/To_4_pre',bbb)
# np.save('/content/drive/My Drive/Newdata/_441to4/To_4_post',ccc)
# np.save('/content/drive/My Drive/Newdata/_441to4/To_4_fu',ddd)

In [8]:
def make_50_times(data):

  tmp = []
  for i in range(24):
    for j in range(50):
      tmp.append(data[i])
  result = np.array(tmp)
  return result

AA = make_50_times(aaa)
BB = make_50_times(bbb)
CC = make_50_times(ccc)
DD = make_50_times(ddd)
print(AA.shape)  # (1200, 4)

(1200, 4)


In [ ]:
# 此cell将 4个label 变成  [4,2]形式   例子 [0 0 1 0] 变成 [1 0 ],[1 0 ], [0 1 ], [1 0]

def from_4_to_2_label(data):
  TMP = []
  for i in range(24):
    Tmp =[]
    for j in range(4):
      tmp = []
      if data[i,j] == 0:
        tmp.append(1) #生成 1,0,0存进list
        tmp.append(0)

        
      elif data[i,j] == 1:
        tmp.append(0)
        tmp.append(1)

      # print(tmp)
      Tmp.append(tmp) 
      # print(len(Tmp))      
    TMP.append(Tmp)
  result = np.array(TMP)
  return result

A = from_4_to_2_label(aaa)
B = from_4_to_2_label(bbb)
C = from_4_to_2_label(ccc)
D = from_4_to_2_label(ddd)
print(A.shape)  # (24, 4, 2)

# np.save('/content/drive/My Drive/Newdata/_441to4/To_16_con',A)
# np.save('/content/drive/My Drive/Newdata/_441to4/To_16_pre',B)
# np.save('/content/drive/My Drive/Newdata/_441to4/To_16_post',C)
# np.save('/content/drive/My Drive/Newdata/_441to4/To_16_fu',D)

(24, 4, 2)


In [ ]:
#该每50个就集成一次 按照病人分类， 24个病人 每个病人150个数据（# pre post fu 每50个 ）
for i in range (24):  
  tmp = np.concatenate((BB[i*50:50*(i+1)],CC[i*50:50*(i+1)],DD[i*50:50*(i+1)]),axis=0)
  np.save('/content/drive/My Drive/Newdata/ordered_by_patient/4/output_150_4_'+str(i+1),tmp)
  # print(i+1)
  # print(tmp.shape)   # (150, 4)


In [ ]:
#该每50个就集成一次 按照病人分类， 24个病人 每个病人200个数据（# con pre post fu 每50个 ）
for i in range (24):  
  tmp = np.concatenate((AA[i*50:50*(i+1)],BB[i*50:50*(i+1)],CC[i*50:50*(i+1)],DD[i*50:50*(i+1)]),axis=0)
  np.save('/content/drive/My Drive/Newdata/ordered_by_patient/4/output_200_4_'+str(i+1),tmp)
  # print(i+1)
  # print(tmp.shape)    # (150, 4)

In [ ]:
patient_order = range(1,25)
order = shuffle(patient_order, random_state = 10)
print(patient_order)
print(order)  
# [14, 18, 20, 5, 2, 22, 19, 15, 23, 17, 24, 13, 11, 6, 9, 7, 1, 3, 8, 12, 21, 10, 16, 4]    random_state =20 
# [6, 2, 8, 3, 12, 13, 22, 14, 15, 4, 23, 7, 19, 11, 21, 24, 9, 20, 17, 18, 1, 16, 5, 10]    random_state =10

range(1, 25)
[6, 2, 8, 3, 12, 13, 22, 14, 15, 4, 23, 7, 19, 11, 21, 24, 9, 20, 17, 18, 1, 16, 5, 10]
